# Инструмент продавца

## Придумать и реализовать прототип аналитического инструмента для продавцов, который поможет им продвигать свои товары на маркетплейсе

$\large{В\: датасете\: {\color{Purple}{WB\_hack \_3 .csv}}\: содержатся\: следующие\: данные:}$

* <span style="color:green"><b>utc_event_time</b></span> – время срабатывание события
* <span style="color:green"><b>utc_event_date</b></span> – дата события
* <span style="color:green"><b>user_id</b></span> – id клиента
* <span style="color:green"><b>city_name</b></span> – город
* <span style="color:green"><b>ecom_event_action</b></span> – действие с товаром
* <span style="color:green"><b>ecom_id</b></span> – id товара
* <span style="color:green"><b>ecom_brand</b></span> – название бренда
* <span style="color:green"><b>ecom_variant</b></span> – вариация товара ( красный/синий, 23/24, 1метр/2метра)
* <span style="color:green"><b>ecom_currency</b></span> – валюта товара
* <span style="color:green"><b>ecom_price100</b></span> – цена товара в копейках
* <span style="color:green"><b>ecom_qty</b></span> – количество товара
* <span style="color:green"><b>ecom_grand_total100</b></span> – сумма товара ( цена*кол-во)
* <span style="color:green"><b>os_manufacturer</b></span> – производитель устройства
* <span style="color:green"><b>device_type</b></span> – тип девайса
* <span style="color:green"><b>traffic_src_kind</b></span> – тип трафика
* <span style="color:green"><b>app_version</b></span> – версия приложения
* <span style="color:green"><b>net_type</b></span> – тип сети
* <span style="color:green"><b>locale</b></span> – локаль

In [209]:
# импортируем библиотеки

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [210]:
# читаем файл

wild_cut = pd.read_csv('wild_cut.csv')

### 1. Чистим данные 

In [211]:
# данные в столбцах ecom_event_action, ecom_id, ecom_brand, ecom_variant, ecom_currency
# нужно преобразовать, а именно убрать вначале и в конце строки символы "['",  "']"

# задаем список названий колонок, в которых будем преобразовывать данные
columns_with_bad_values = ['ecom_event_action', 'ecom_id', 'ecom_brand',
                           'ecom_variant', 'ecom_currency']

In [212]:
# с помощью регулярных выражений убираем лишнее из столбцов

wild_cut[columns_with_bad_values] = wild_cut[columns_with_bad_values] \
                              .apply(lambda x: x.str.extract(r'(\w+)', expand=False))

In [213]:
# как видим, все получилось

wild_cut.head()

,Unnamed: 0,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price100,ecom_qty,ecom_grand_total100,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,0,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,[248500],[1],[248500],Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,1,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,[75400],[1],[75400],samsung,smartphone,mobile,4.2.7000,3G,ru
2,2,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,[74500],[1],[74500],samsung,smartphone,mobile,4.2.7000,3G,ru
3,3,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,RUB,[131900],[1],[131900],samsung,smartphone,mobile,4.3.1003,4G,ru
4,4,2021-10-21 19:43:45,2021-10-21,12954672620163632510,Москва,remove_from_cart,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,[1077100],[1],[1077100],samsung,smartphone,mobile,4.3.5000,Wi-Fi,ru


In [214]:
# делаем то же самое для столбцов ecom_price100, ecom_qty, ecom_grand_total100,
# только приводим их к типу int32

columns_to_int32 = ['ecom_price100', 'ecom_qty', 'ecom_grand_total100']

In [215]:
# пользуемся регулярным выражением

wild_cut[columns_to_int32] = wild_cut[columns_to_int32] \
                              .apply(lambda x: x.str.extract(r'(\d+)', expand=False)) \
                              .astype(dtype='int32')

In [216]:
# переведем цену товара и сумму товара в рубли

wild_cut['ecom_price100'] = wild_cut['ecom_price100'] / 100
wild_cut['ecom_grand_total100'] = wild_cut['ecom_grand_total100'] / 100

In [217]:
# переименуем колонки

wild_cut = wild_cut.rename(columns={'ecom_grand_total100': 'ecom_grand_total', 
                                    'ecom_price100': 'ecom_price'})

In [218]:
# посмотрим, какие типы данных у нас есть

wild_cut.dtypes

Unnamed: 0             int64
utc_event_time        object
utc_event_date        object
user_id               uint64
city_name             object
ecom_event_action     object
ecom_id               object
ecom_brand            object
ecom_variant          object
ecom_currency         object
ecom_price           float64
ecom_qty               int32
ecom_grand_total     float64
os_manufacturer       object
device_type           object
traffic_src_kind      object
app_version           object
net_type              object
locale                object
dtype: object

In [219]:
# здесь уместно перевести колонки utc_event_time и utc_event_date в 
# формат datetime

wild_cut[['utc_event_time']] = wild_cut[['utc_event_time']] \
                                .apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')

wild_cut[['utc_event_date']] = wild_cut[['utc_event_date']] \
                                .apply(pd.to_datetime, format='%Y-%m-%d')

# можно было просто написать pd.to_datetime(wild_cut.utc_event_time, 
#                                           format = '%Y-%m-%d %H:%M:%S'),
# но используем общую формулу для нескольких столбцов

In [220]:
# мы почистили данные, и можем с ними работать дальше

wild_cut.head()

,Unnamed: 0,utc_event_time,utc_event_date,user_id,city_name,ecom_event_action,ecom_id,ecom_brand,ecom_variant,ecom_currency,ecom_price,ecom_qty,ecom_grand_total,os_manufacturer,device_type,traffic_src_kind,app_version,net_type,locale
0,0,2021-10-01 19:40:22,2021-10-01,11090758688966051676,Могилёв,add_to_cart,BEFC96A9D6DEA3A9817BC5E5157B0A1C,6B0630D20C913739C8164D8208A1B13D,3B9A6121A509CBADC049AEABD3ED0223,RUB,2485.0,1,2485.0,Xiaomi,smartphone,mobile,4.3.4000,Wi-Fi,ru
1,1,2021-10-01 07:50:59,2021-10-01,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,754.0,1,754.0,samsung,smartphone,mobile,4.2.7000,3G,ru
2,2,2021-10-02 11:51:12,2021-10-02,12487687213536341095,Самара,view_item,97F9B1135CF24F4CFFF4664435AEC2B9,FA74FF059207A1569295D3E83CA88E53,3B9A6121A509CBADC049AEABD3ED0223,RUB,745.0,1,745.0,samsung,smartphone,mobile,4.2.7000,3G,ru
3,3,2021-10-03 15:06:38,2021-10-03,10383386994170804641,Москва,view_item,7D9A98C6E8BF033613B74252D9D24B33,681425F1391C8CA4026B28066C7B3B4A,3B9A6121A509CBADC049AEABD3ED0223,RUB,1319.0,1,1319.0,samsung,smartphone,mobile,4.3.1003,4G,ru
4,4,2021-10-21 19:43:45,2021-10-21,12954672620163632510,Москва,remove_from_cart,B0270133170E1B886F4FA77B7E1F0EB9,7921BD71B70D139454324A845115B9E3,50D05F0ABB80F2B62C5F67F057E88D5F,RUB,10771.0,1,10771.0,samsung,smartphone,mobile,4.3.5000,Wi-Fi,ru


### 2. Определимся с метриками

#### 2.1 Разделим товары по 3 группам:
#### I группа -> приносящие прибыль 80%
#### II группа -> приносящие прибыль 15%
#### III группа -> приносящие прибыль 5% 

In [228]:
# сгруппируем товары по ecom_id, 
# и разобьем по группам

all_sum_ecom = wild_cut.ecom_grand_total.sum()

our_percent = wild_cut.groupby('ecom_id') \
                      .agg({'ecom_grand_total': 'sum'}) \
                      .rename(columns={'ecom_grand_total': 'percent_sum_grand_total'}) \
                      .sort_values(by='percent_sum_grand_total',
                                   ascending=False) / all_sum_ecom * 100

In [258]:
# добавляем в списки товары каждой группы

group_1 = []
group_2 = []
group_3 = []

lim = 0
for ecom_id, percent in zip(our_percent.index, our_percent.percent_sum_grand_total):
        if lim < 80:
            group_1.append(ecom_id)
            lim += percent
        elif lim < 95:
            group_2.append(ecom_id)
            lim += percent
        else:
            group_3.append(ecom_id)

In [260]:
# выводим продавцу наименования товаров:
# I группа -> приносит 80% прибыли;
# II группа -> приносит 15% прибыли;
# III группа -> приносит 5% прибыли -> рекомендуем от продажи
#                                      этих товаров отказаться
#                                      или пересмотреть их параметры
#                                      (описание, цвет и т.д.)

print('I группа: ', group_1, '\n\n',
      'II группа: ', group_2, '\n\n',
      'III группа: ', group_3, '\n\n')

I группа:  ['B0270133170E1B886F4FA77B7E1F0EB9', 'BDBD23C5359EF7B3F85763C032892B85', '8360E6D158AEBD26BBBE835C38F88748'] 

 II группа:  ['AC688D3B8E8D812E2B49496BE59286E3', '97F9B1135CF24F4CFFF4664435AEC2B9', 'ECD8847A3E389369BA25918DFF9B5B83', 'AF51FBD3B09A5BE324C76C524118BFD3', '6050B6B1F05ABDB4B33DA4BCEB46DE9B'] 

 III группа:  ['BF4304D1BE666DB38DF823DE35823D4D', '61F3488FEF9234C5B0E9B68CAF0AB51F', '7D9A98C6E8BF033613B74252D9D24B33', 'BEFC96A9D6DEA3A9817BC5E5157B0A1C', '5C8925190BE8C493517153B561BA21E7', 'A6FC1D41C2A36CD6ED1428BC40991F3A', '484FD293DAE723EB16AF9C7548C85ED9', '30DCD4FD8AB5E0ED4706DC0A19F5C7A9', '02C451A73FDD41CDD05F1E33E4BC60FC', 'B47F278C96726C7B5022C3007EB0A6C9', 'C0C9F2DC21904776545FFBB2FC3D646C', '67FC8BB094EC6205479255C1F8028E20', 'CA2AC4DF8A5C07E398DB8584A23D57F8', '2AA18144D676AC3D04B5250B416DEB64', '0F49D92CEA8C80166B0605ED3D1A8E5D', '6D88AAEEAB611F290D931ACCC12045CB', 'C1506A9C4FBD8AD89DA9FA0CC6616566', '6BDA30FE61747A5C2FA82F13843104A8', 'B3A955CA48D147C9C9

#### 2.2 Посчитаем конверсию для каждого товара
#### (купил / положил в корзину)

In [261]:
# сначала посмотрим действия, которые совершаются с товарами

wild_cut.ecom_event_action.unique()

# add_to_cart -> добавлено в корзину
# view_item -> просмотрен товар
# remove_from_cart -> удалено из корзины
# begin_checkout -> начато офрмление заказа
# purchase -> покупка

array(['add_to_cart', 'view_item', 'remove_from_cart', 'begin_checkout',
       'purchase'], dtype=object)

In [372]:
# отберем товары с ecom_event_action = ['add_to_cart', 'purchase']
# и сгруппируем их

product_conversion = wild_cut[[
                      'ecom_id',
                      'ecom_event_action',
                      'user_id']].query("ecom_event_action == ['add_to_cart','purchase']") \
                                 .groupby(['ecom_id', 'ecom_event_action']) \
                                 .agg({'user_id': 'count'}) \
                                 .rename(columns={'user_id': 'count_action'}) \
                                 .unstack(level='ecom_event_action', fill_value=0) \
                                 ['count_action'] \
                                 .reset_index() \
                                 .query('add_to_cart > 0')
        

In [373]:
product_conversion.head()

ecom_event_action,ecom_id,add_to_cart,purchase
0,02C451A73FDD41CDD05F1E33E4BC60FC,1,0
1,0D09B305F4E0550B710DFB0ADB496070,3,0
2,0F49D92CEA8C80166B0605ED3D1A8E5D,8,3
3,1C0662D8F63AA9CD3B9DB37506F19BE0,1,0
4,3AD6487EBF165BDE4EECE355B64A80A8,1,0


In [374]:
product_conversion['conversion'] = (product_conversion['purchase']
                                    / product_conversion['add_to_cart'] * 100)

In [375]:
product_conversion.sort_values(by='conversion', ascending=False)

ecom_event_action,ecom_id,add_to_cart,purchase,conversion
33,C0C9F2DC21904776545FFBB2FC3D646C,11,11,100.000000
22,A6FC1D41C2A36CD6ED1428BC40991F3A,11,9,81.818182
32,BF4304D1BE666DB38DF823DE35823D4D,9,6,66.666667
12,6050B6B1F05ABDB4B33DA4BCEB46DE9B,5,3,60.000000
2,0F49D92CEA8C80166B0605ED3D1A8E5D,8,3,37.500000
34,C1506A9C4FBD8AD89DA9FA0CC6616566,3,1,33.333333
11,5C8925190BE8C493517153B561BA21E7,6,2,33.333333
20,97F9B1135CF24F4CFFF4664435AEC2B9,21,7,33.333333
30,BDBD23C5359EF7B3F85763C032892B85,12,3,25.000000
17,7D9A98C6E8BF033613B74252D9D24B33,4,1,25.000000
